In [1]:

from google.colab import files


In [2]:
# uploading the data into df
uploaded = files.upload()

Saving gas_turbines.csv to gas_turbines.csv


In [4]:

import pandas as pd
import io
  
df = pd.read_csv(io.BytesIO(uploaded['gas_turbines.csv' ]))
df.head()

,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY,CDP,CO,NOX
0,6.8594,1007.9,96.799,3.5000,19.663,1059.2,550.00,114.70,10.605,3.1547,82.722
1,6.7850,1008.4,97.118,3.4998,19.728,1059.3,550.00,114.72,10.598,3.2363,82.776
2,6.8977,1008.8,95.939,3.4824,19.779,1059.4,549.87,114.71,10.601,3.2012,82.468
3,7.0569,1009.2,95.249,3.4805,19.792,1059.6,549.99,114.72,10.606,3.1923,82.670
4,7.3978,1009.7,95.150,3.4976,19.765,1059.7,549.98,114.72,10.612,3.2484,82.311


In [5]:
df.shape

(15039, 11)

In [ ]:
# CDP, CO,NOX are outlet parameters which depends on input parameters combustion so we need not to use them

In [6]:
# selecting required columns in to dataset
dataset = df.iloc[:, 0:8].copy()
dataset.head()

,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY
0,6.8594,1007.9,96.799,3.5000,19.663,1059.2,550.00,114.70
1,6.7850,1008.4,97.118,3.4998,19.728,1059.3,550.00,114.72
2,6.8977,1008.8,95.939,3.4824,19.779,1059.4,549.87,114.71
3,7.0569,1009.2,95.249,3.4805,19.792,1059.6,549.99,114.72
4,7.3978,1009.7,95.150,3.4976,19.765,1059.7,549.98,114.72


In [7]:
# defining imput and target variables
X = dataset.iloc[:,0:7]
y = pd.DataFrame(dataset.iloc[:,7])

In [10]:
# Importing the necessary  packages for scaling the data
import pandas as pd
import numpy as np
import keras
from sklearn.preprocessing import StandardScaler

In [14]:
# Standardization of X variable and transforming y into log
# after many tries it is f=giving better result
a = StandardScaler()
#a.fit(X_train)
#X_standardized = a.fit_transform(X_train)
#y_standardized = a.fit_transform(y_train)

X_standardized = a.fit_transform(X)
y_standardized = np.log(y)

In [15]:
# splitting the data into test and train
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_standardized,y_standardized,test_size=0.2,random_state=7)

In [16]:
pd.DataFrame(X_train).describe()

,0,1,2,3,4,5,6
count,12031.000000,12031.000000,12031.000000,12031.000000,12031.000000,12031.000000,12031.000000
mean,-0.004144,-0.000643,0.004415,0.000224,-0.001218,-0.004496,-0.003389
std,0.997276,0.998345,0.998198,1.003430,1.004428,1.005130,1.004180
min,-2.276462,-4.266288,-3.536594,-2.779497,-1.806771,-5.021933,-4.188141
25%,-0.839757,-0.655052,-0.667744,-0.631692,-0.518370,-0.287330,-0.418377
50%,0.051969,-0.062279,0.230394,-0.017620,-0.082434,0.290504,0.571257
75%,0.792522,0.577292,0.792673,0.463259,0.444667,0.750350,0.591596
max,2.266234,3.244771,1.528011,4.486233,2.856152,1.028678,0.656428


#### Creating Model


# As the output data is contineous variable not categorical we will use regression 

In [17]:
# Importing the necessary packages
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import Adam

In [18]:
# creating the  model
def create_model():
    model = Sequential()
    model.add(Dense(7, input_dim=7, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='relu'))
    
    adam=Adam(lr=0.3)
    model.compile(loss= 'MeanSquaredError', optimizer=adam, metrics=['MeanSquaredError'])
    return model

    # we use all parameters related to regression like loss functions, metrics and others
    # number of layers and hidden layers etc also can be varried
    # Here we are creating simple model

In [20]:
# Assigning the regressor to the mymodel

mymodel = KerasRegressor(build_fn = create_model,verbose = 0)


# Defining the grid search parameters
# we only grid search for batch size and epochs first 
# the same can be followed for other parameters

batch_size = [1000]
epochs = [10]


# Combining the parameters into dictionery

param_grid = dict(batch_size = batch_size,epochs = epochs)


# Fitting the gird with train data
grid = GridSearchCV(estimator = mymodel,param_grid = param_grid,cv = 5,verbose = 2)
grid_result = grid.fit(X_train,y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] batch_size=1000, epochs=10 ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ....................... batch_size=1000, epochs=10, total=   1.3s
[CV] batch_size=1000, epochs=10 ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s remaining:    0.0s


[CV] ....................... batch_size=1000, epochs=10, total=   0.6s
[CV] batch_size=1000, epochs=10 ......................................
[CV] ....................... batch_size=1000, epochs=10, total=   0.6s
[CV] batch_size=1000, epochs=10 ......................................
[CV] ....................... batch_size=1000, epochs=10, total=   0.6s
[CV] batch_size=1000, epochs=10 ......................................
[CV] ....................... batch_size=1000, epochs=10, total=   0.6s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    3.9s finished


In [21]:
# finding best parameters from the grid search

print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : -0.00497392249526456, using {'batch_size': 1000, 'epochs': 10}
-0.00497392249526456,0.005112784086769297 with: {'batch_size': 1000, 'epochs': 10}


In [25]:
# now we will fit the data to into model with parameters obtained from grid search

mymodel.fit(X_train, y_train, batch_size=1000, epochs=10)

In [27]:
# preding the y from x test data

y_pr = mymodel.predict(X_test)

In [28]:
y_pr

array([5.0671644, 5.0331125, 5.01749  , ..., 4.8264847, 4.892634 ,
       5.121421 ], dtype=float32)

In [29]:
# finding the rmse score for the predcted data
# the rmse score seems good

from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pr,  squared=False)

0.01197138959639003

In [31]:
# Calculating the rsquare for finding goodness of fit

from sklearn.metrics import r2_score
r2_score(y_test, y_pr)

0.9894755394664212

In [ ]:
# the R square  value also seems good 

#### Same way we can tune for other hyoer parameters

# a caricature of the tuning of all hyper parametrs is given below
# but it takes long time and high memory fot running the code 
# so we skip this step as we already good good fit
# ofcourse we can try other nethods also to check
# as i have done already with other parameters 


In [ ]:
def create_model(learning_rate,dropout_rate,activation_function,init,neuron1,neuron2, loss_func ,optima, metrico ):
    model = Sequential()
    model.add(Dense(neuron1,input_dim = 8,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neuron2,input_dim = neuron1,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1,activation = 'linear'))
    
    adam = Adam(lr = learning_rate)
    model.compile(loss = loss_func ,optimizer = optima ,metrics = metrico)
    return model

# Create the model

model = KerasRegressor(build_fn = create_model,verbose = 2)

# Define the grid search parameters

batch_size = [10,20,40]
epochs = [10,50,100]
learning_rate = [0.001,0.01,0.1]
dropout_rate = [0.0,0.1,0.2]
activation_function = ['softmax','relu','tanh','linear']
init = ['uniform','normal','zero']
neuron1 = [4,8,16]
neuron2 = [2,4,8]
loss_func =  [regressor loss functions]
optima = ['SGD', "Adam"....]
metrico = [regressor metrics]



param_grids = dict(batch_size = batch_size,epochs = epochs,learning_rate = learning_rate,dropout_rate = dropout_rate,
                   activation_function = activation_function,init = init,neuron1 = neuron1,neuron2 = neuron2, optima = optima, 
                   metrico = metrico, loss_func = lossfunc)



grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_standardized,y)


print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

In [ ]:
# from the above we can fine tune model and find best fits
